# Overview

This notebook is intended for the purpose of Baseline Modelling and Feature Engineering

In [2]:
import pandas as pd
import numpy as np
from time import time
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import warnings

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
data = pd.read_csv('~/w266_project_topmusichits/data/interim/songs_charts_new_2000s.csv',index_col=[0])

In [3]:
data.head(50)

,artist_name,lyrics,album_name,genre,subgenre,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,song_year,trending_year,weeks_on_chart,hit
song_name,,,,,,,,,,,,,,,,,,,,,
?,MF DOOM,NA My servants began to forge what was to beco...,Operation: Doomsday (Complete),rap,gangster rap,0.532,0.629,10,-7.955,0,...,0.260000,0.000000,0.4780,0.5450,94.030,188893,2011,0,0,0
0 To 100 / The Catch Up,Drake,NA NA Fuck bein' on some chill shit We go 0 to...,0 To 100 / The Catch Up,r&b,urban contemporary,0.559,0.716,11,-3.964,0,...,0.737000,0.000000,0.2520,0.5040,176.618,275227,2013,2014,20,1
10%,KAYTRANADA,You keep on takin' from me But where's my ten ...,BUBBA,r&b,urban contemporary,0.794,0.757,6,-6.644,0,...,0.026700,0.000306,0.0621,0.6150,107.990,186680,2019,0,0,0
1000 Nights,FRENSHIP,When the fever's gone And the road's to dust I...,1000 Nights,pop,indie poptimism,0.594,0.709,6,-9.541,0,...,0.031500,0.056200,0.0738,0.5870,90.001,240293,2017,0,0,0
1000 Rounds,Pouya,"Bullets fly, legs buckle, bodies pile to the s...",1000 Rounds,rap,gangster rap,0.825,0.761,8,-5.389,1,...,0.011100,0.003590,0.3340,0.1610,149.970,163371,2017,0,0,0
"100 Bands (feat. Quavo, 21 Savage, YG & Meek Mill)",Mustard,"Whoop, woo!Y'all already know who I am? Right?...","100 Bands (feat. Quavo, 21 Savage, YG & Meek M...",r&b,urban contemporary,0.871,0.493,9,-6.775,1,...,0.010400,0.000000,0.0917,0.1570,93.016,179388,2019,0,0,0
100 Miles and Running (ft. Wale & John Lindahl),Logic,"Woo, woo, yeah Alright, here we go Yeah yeah, ...",YSIV,rap,hip hop,0.835,0.929,7,-3.595,1,...,0.579000,0.000001,0.3800,0.9600,119.047,353453,2018,0,0,0
100 Shooters (feat. Meek Mill),Future,"Tay Keith, fuck these niggas up Yeah, The Wizr...",100 Shooters (feat. Meek Mill),rap,hip hop,0.645,0.675,1,-5.282,0,...,0.010400,0.000000,0.2730,0.1710,120.672,209120,2019,0,0,0
101,WALLA,Don't you know about this road of rust? People...,101,pop,indie poptimism,0.710,0.827,0,-5.244,1,...,0.043700,0.000013,0.0896,0.8330,110.003,203500,2015,0,0,0


# STEPS TO BUILD BASELINE MODEL

## EDA
- Missing and Null Values for relevant data columns
- Distribution of different genres, years, language, hits vs non-hits - if disproprotionate lets sample X random songs from each genre
- Trends in word counts, unique word counts, meta data in hit vs. non-hit songs
- Top words or Word clouds
- Song counts per genre, year etc.

First step: Choosing English songs as would not be able to accurately process other languages. Some reasons below:
- Words in other languages may be spelled identically to certain English words and then would be categorized as the same word, which may mislead the analysis.
- A word with the same meaning but in two different languages could be categorized as two different words. This is misleading as they should be considered the same.
- Stop words in other languages might be an issue

In [4]:
data.shape


(8748, 21)

In [5]:
#Missing data (looks good)
data.isnull().sum()

artist_name         0
lyrics              0
album_name          0
genre               0
subgenre            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
song_year           0
trending_year       0
weeks_on_chart      0
hit                 0
dtype: int64

In [10]:
#drop duplicates
#data.drop_duplicates()
#data.drop_duplicates(subset=['track_name', 'track_artist'], keep='last')
#data.drop_duplicates(subset=['lyrics'])
#duplicateRowsDF = data[data.duplicated(subset=['lyrics'],keep='first')]
#duplicateRowsDF = data[data.duplicated(['lyrics'])]
#data.shape
#dataframe.loc[dataframe['Percentage'] > 80]
#duplicateRowsDF.loc[duplicateRowsDF['track_name']=="pony"]
#duplicateRowsDF['track_name'].value_counts()
#duplicateRowsDF

In [6]:
data.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,song_year,trending_year,weeks_on_chart,hit
count,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000,8748.000000
mean,0.631792,0.672050,5.294124,-6.577671,0.589163,0.111889,0.189103,0.036530,0.186723,0.492400,120.477306,224396.713877,2013.622314,406.544239,4.007773,0.202332
std,0.152173,0.186045,3.610260,2.628323,0.492014,0.107732,0.236353,0.136687,0.146950,0.220673,28.855042,52493.095097,5.535156,807.268420,9.927895,0.401761
min,0.116000,0.016700,0.000000,-26.279000,0.000000,0.022800,0.000001,0.000000,0.018900,0.000010,46.169000,37640.000000,2000.000000,0.000000,0.000000,0.000000
25%,0.530000,0.546000,2.000000,-7.923750,0.000000,0.039800,0.015100,0.000000,0.096200,0.326000,97.318750,191655.750000,2010.000000,0.000000,0.000000,0.000000
50%,0.643000,0.690000,5.000000,-6.163500,1.000000,0.062050,0.084450,0.000004,0.126000,0.486000,119.845000,217660.000000,2016.000000,0.000000,0.000000,0.000000
75%,0.743000,0.818250,8.000000,-4.747750,1.000000,0.143000,0.277000,0.000768,0.235000,0.659000,139.934000,249630.250000,2018.000000,0.000000,0.000000,0.000000
max,0.979000,1.000000,11.000000,0.551000,1.000000,0.918000,0.992000,0.974000,0.994000,0.991000,210.164000,516760.000000,2020.000000,2021.000000,87.000000,1.000000


## Baseline Model

- Remove stop words? Yes
- Do we stem or lemmatize our text data, or leave the words as is? No
- Is basic tokenization enough, or do we need to support special edge cases through the use of regex? No
- Do we stick with English words only or do we allow for other languages? All words
- Do we use the entire vocabulary, or just limit the model to a subset of the most frequently used words? If so, how many? entire voacab for now
- Do we engineer other features, such as bigrams, POS tags, or Mutual Information Scores? after baseline
- What sort of vectorization should we use in our model? Boolean Vectorization? Count Vectorization? TF-IDF? More advanced vectorization strategies such as Word2Vec? TF-IDF for baseline
- Prorpotion of hits in the train and test set??
- How are we splitting this data based on time series- (cutoff date question for Mark)

## Splitting into train and test set

In [4]:
#define cutoff date
split_year = 2017
train_df = data[data['song_year'] < split_year]
test_df = data[data['song_year'] >= split_year]

#create test and train sets
count_train = train_df['song_year'].value_counts()
count_test = test_df['song_year'].value_counts()

#confirm correctly split
print("Year counts in train set:\n", count_train)
print("Year counts in test set:\n", count_test)

train_data = train_df['lyrics']
train_labels = train_df['hit']
test_data = test_df['lyrics']
test_labels = test_df['hit']

print("Confrim total adds up to initital total of 8748 records:", train_data.shape[0] + test_data.shape[0])


Year counts in train set:
 2016    762
2015    626
2014    475
2013    344
2012    303
2011    294
2008    293
2005    256
2010    231
2006    212
2007    205
2009    203
2004    199
2003    188
2001    184
2000    162
2002    141
Name: song_year, dtype: int64
Year counts in test set:
 2019    1755
2018     908
2017     843
2020     164
Name: song_year, dtype: int64
Confrim total adds up to initital total of 8748 records: 8748


In [5]:
#ONLY FOR RANDOM SPLIT WHICH WE WILL NOT USE WITH TIME SERIES DATA
#from sklearn.model_selection import train_test_split

# Extracting text from df (x) and outcome y

# x = data['lyrics']
# y = data['Hit']

#split ratio
# train_ratio = 0.70
# validation_ratio = 0.15
# test_ratio = 0.15

#train is now 75% of the entire data set
#train_data, test_data, train_labels, test_labels = train_test_split(x, y, test_size=1 - train_ratio, random_state=1)

#test is now 10% of the initial data set
#validation is now 15% of the initial data set
#dev_data, test_data, dev_labels, test_labels = train_test_split(test_data, test_labels, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1) 

#print(X_train, X_dev, X_test)
#print(train_data.shape)


### TF-IDF countvectorizer
TF = (Number of times term t appears in a document) / (Total number of terms in the document).
IDF = log_e(Total number of documents / Number of documents with term t in it)

In [6]:
t = time() 

#Using TFIDF to convert into bag of words
tf_vectorizer = TfidfVectorizer(stop_words='english')
#ct_vectorizer = CountVectorizer() # same results with just countvectorizer

X_train_tf = tf_vectorizer.fit_transform(train_data)

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 1.376966 seconds
n_samples: 5078, n_features: 28916


In [7]:
t = time()
X_test_tf = tf_vectorizer.transform(test_data)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 1.097617 seconds
n_samples: 3670, n_features: 28916


### Smoothening and Final Baseline Model

In [8]:
# summarize original class distribution
print("Before oversampling: ",Counter(train_labels))

# define oversampling strategy
SMOTE = SMOTE()

# fit and apply the transform
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train_tf, train_labels)

# summarize class distribution
print("After oversampling: ",Counter(y_train_SMOTE))


Before oversampling:  Counter({0: 3798, 1: 1280})
After oversampling:  Counter({0: 3798, 1: 3798})


In [19]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = MultinomialNB()
#model = LogisticRegression(penalty='l2',max_iter= 1000)
#model = RandomForestClassifier(n_estimators=100)
#model = DecisionTreeClassifier(random_state=2)
#model = SVC(probability=True, kernel='rbf')
#model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of NAIVE BAYES:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix NAIVE BAYES:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.6233699140033371
accuracy of NAIVE BAYES:   0.513
              precision    recall  f1-score   support

           0       0.93      0.47      0.63      3180
           1       0.18      0.77      0.30       490

    accuracy                           0.51      3670
   macro avg       0.56      0.62      0.46      3670
weighted avg       0.83      0.51      0.58      3670

confusion matrix NAIVE BAYES:
[[1505 1675]
 [ 111  379]]
------------------------------


In [21]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = LogisticRegression(penalty='l2',max_iter= 1000)


clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of Logistic Regression:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix LOGISTIC REGRESSION:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.5987934796560134
accuracy of Logistic Regression:   0.646
              precision    recall  f1-score   support

           0       0.90      0.66      0.76      3180
           1       0.20      0.53      0.29       490

    accuracy                           0.65      3670
   macro avg       0.55      0.60      0.53      3670
weighted avg       0.81      0.65      0.70      3670

confusion matrix LOGISTIC REGRESSION:
[[2108 1072]
 [ 228  262]]
------------------------------


In [22]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = RandomForestClassifier(n_estimators=100)
#model = DecisionTreeClassifier(random_state=2)
#model = SVC(probability=True, kernel='rbf')
#model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of RANDOM FOREST:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix RANDOM FOREST:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.5747368758824284
accuracy of RANDOM FOREST:   0.824
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      3180
           1       0.30      0.23      0.26       490

    accuracy                           0.82      3670
   macro avg       0.59      0.57      0.58      3670
weighted avg       0.81      0.82      0.81      3670

confusion matrix RANDOM FOREST:
[[2909  271]
 [ 375  115]]
------------------------------


In [17]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = DecisionTreeClassifier(random_state=2)
#model = SVC(probability=True, kernel='rbf')
#model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of DecisionTree:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix DecisionTree:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.5755775895263766
accuracy of DecisionTree:   0.656
              precision    recall  f1-score   support

           0       0.89      0.69      0.78      3180
           1       0.19      0.47      0.27       490

    accuracy                           0.66      3670
   macro avg       0.54      0.58      0.52      3670
weighted avg       0.80      0.66      0.71      3670

confusion matrix DecisionTree:
[[2181  999]
 [ 262  228]]
------------------------------


In [18]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = SVC(probability=True, kernel='rbf')
#model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of DecisionTree:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix DecisionTree:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.5243742780130921
accuracy of DecisionTree:   0.838
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      3180
           1       0.24      0.10      0.14       490

    accuracy                           0.84      3670
   macro avg       0.56      0.52      0.52      3670
weighted avg       0.79      0.84      0.81      3670

confusion matrix DecisionTree:
[[3030  150]
 [ 443   47]]
------------------------------


In [9]:
#PART 2
# import SVM libraries 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

#no smoothening of the test data
model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

clf_SMOTE = model.fit(X_train_SMOTE, y_train_SMOTE)
pred_SMOTE = clf_SMOTE.predict(X_test_tf)

print("ROC AUC score for oversampled SMOTE data: ", roc_auc_score(test_labels, pred_SMOTE))

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, pred_SMOTE)
print("accuracy of MLP:   %0.3f" % score1)

print(metrics.classification_report(test_labels, pred_SMOTE))

print("confusion matrix MLP:")
print(metrics.confusion_matrix(test_labels, pred_SMOTE))

print('------------------------------')

ROC AUC score for oversampled SMOTE data:  0.5290976768065717
accuracy of MLP:   0.706
              precision    recall  f1-score   support

           0       0.88      0.77      0.82      3180
           1       0.16      0.29      0.21       490

    accuracy                           0.71      3670
   macro avg       0.52      0.53      0.51      3670
weighted avg       0.78      0.71      0.74      3670

confusion matrix MLP:
[[2450  730]
 [ 349  141]]
------------------------------


# APPENDIX

### Unsmoothned models

In [16]:
#Unsmoothned models
# build naive bayes classification model
t = time()

naive_bayes_classifier = MultinomialNB()
#naive_bayes_classifier.fit(X_train_tf, train_labels)


training_time = time() - t
print("train time: %0.3fs" % training_time)

# predict Hit vs. non-Hit from the testing dataset
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

train time: 0.393s
test time:  0.001s
accuracy:   0.899
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5495
           1       0.07      0.02      0.03       490

    accuracy                           0.90      5985
   macro avg       0.49      0.50      0.49      5985
weighted avg       0.85      0.90      0.87      5985

confusion matrix:
[[5370  125]
 [ 481    9]]
------------------------------


In [27]:
#Logistic Regression

# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(penalty='l2',max_iter= 1000)

# fit the model with data
logreg.fit(X_train_tf,train_labels)

#test on test data
y_pred = logreg.predict(X_test_tf)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

accuracy:   0.899
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5495
           1       0.07      0.02      0.03       490

    accuracy                           0.90      5985
   macro avg       0.49      0.50      0.49      5985
weighted avg       0.85      0.90      0.87      5985

confusion matrix:
[[5370  125]
 [ 481    9]]
------------------------------


In [28]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_tf,train_labels)

y_pred=clf.predict(X_test_tf)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

accuracy:   0.914
              precision    recall  f1-score   support

           0       0.92      1.00      0.95      5495
           1       0.00      0.00      0.00       490

    accuracy                           0.91      5985
   macro avg       0.46      0.50      0.48      5985
weighted avg       0.84      0.91      0.88      5985

confusion matrix:
[[5469   26]
 [ 490    0]]
------------------------------


In [29]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier
import time
start_time = time.time()

# Initialize the model
clf = DecisionTreeClassifier(random_state=2)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_tf,train_labels)

y_pred=clf.predict(X_test_tf)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

accuracy:   0.723
              precision    recall  f1-score   support

           0       0.92      0.77      0.84      5495
           1       0.09      0.25      0.13       490

    accuracy                           0.72      5985
   macro avg       0.50      0.51      0.48      5985
weighted avg       0.85      0.72      0.78      5985

confusion matrix:
[[4205 1290]
 [ 369  121]]
------------------------------


In [30]:
#SVM
from sklearn.svm import SVC

# Initialize the model
clf = SVC(probability=True, kernel='rbf')

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_tf,train_labels)

y_pred=clf.predict(X_test_tf)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

accuracy:   0.913
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      5495
           1       0.03      0.00      0.00       490

    accuracy                           0.91      5985
   macro avg       0.47      0.50      0.48      5985
weighted avg       0.85      0.91      0.88      5985

confusion matrix:
[[5464   31]
 [ 489    1]]
------------------------------


In [33]:
from sklearn.neural_network import MLPClassifier
# Initialize the model
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_tf,train_labels)

y_pred=clf.predict(X_test_tf)

# compute the performance measures
score1 = metrics.accuracy_score(test_labels, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(test_labels, y_pred))

print("confusion matrix:")
print(metrics.confusion_matrix(test_labels, y_pred))

print('------------------------------')

accuracy:   0.704
              precision    recall  f1-score   support

           0       0.92      0.74      0.82      5495
           1       0.09      0.28      0.13       490

    accuracy                           0.70      5985
   macro avg       0.50      0.51      0.48      5985
weighted avg       0.85      0.70      0.77      5985

confusion matrix:
[[4079 1416]
 [ 354  136]]
------------------------------


## Evaluation Metrics
- Fail to predict the hit (percision) or falsely predict hit (recall) TP/TP+FN
- Falsely predicting is waste of resources for companies, failing to predict can be opportunity cost- F1-score

## Data Cleaning
- Tokenize sentences
- Remove capital letters
- Remove stopwords
- Remove non-alphanumeric characters
- Lemmatize the tokens

In [13]:
#Helper function to accomplish the tasks above
def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    return clean_text

In [16]:
import re
data['clean']=data['lyrics'].map(clean)
data['lyrics_clean']=data['clean'].apply(lambda x: " ".join([str(word) for word in x]))

In [17]:
print(data['lyrics'].iloc[1])
print("\n--------CLEAN--------------\n")
print(data['lyrics_clean'].iloc[1])

NA Yeah, Spyderman and Freeze in full effect Uh-huh You ready, Ron? I'm ready You ready, Biv? I'm ready, Slick, are you? Oh, yeah, break it down NA Girl, I, must (warn you) I sense something strange in my mind Situation is (serious) Let's cure it cause we're running out of time It's oh, so (beautiful) Relationships they seem from the start It's all so (deadly) When love is not together from the heart It's drivin' me out of my mind! That's why it's HARD for me to find Can't get it out of my head! Miss her, kiss her, love her(Wrong move you're dead!) That girl is (poison)...Never trust a big butt and smile That girl is (poison)..("POISON!!") NA (-caution) Before I start to meet a fly girl, you know? Cause in some (portions) You'll think she's the best thing in the world She's so - (fly) She'll drive you right out of your mind And steal your heart when you're blind Beware she's schemin', she'll make you think you're dreamin' YOU'LL fall in love and you'll be screamin', demon, HOO.. Poison

### More robust Clean data

In [29]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [31]:
def clean_text(text):
    '''Text Preprocessing '''
    
    # Convert words to lower case
    text = text.lower()
    
    # Expand contractions
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
#     # remove stopwords
#     if remove_stopwords:
#         text = text.split()
#         stops = set(stopwords.words("english"))
#         text = [w for w in text if not w in stops]
#         text = " ".join(text)

    # Tokenize each word
    text =  nltk.WordPunctTokenizer().tokenize(text)
    
    # Lemmatize each token
    lemm = nltk.stem.WordNetLemmatizer()
    text = list(map(lambda word:list(map(lemm.lemmatize, word)), text))
    
    return text

## Balancing DataSet

## Feature Engineering 

### Sentiment Score or Sentiment itself
### Add Decade or 5 year column
### Word frequency: number of words per song
### Word Length: average lenght of individual words in a text
### Lexical diversity: Number of unique words used in a text (song vocabulary)
### Lexical Density: The number of unique words divided by the total number of words (word repition)
### Presence or Number of repeated contigous phrases
### Number of bad or profane words
### Number of words on female profanity
### Number of keywords or topics (can get from LDA or K-clustering or non-matrix factorization, or wordclouds)
### Cosine Similarity Scores between songs 
### Part of Speech analysis -or Speech tagging
### building n-grams
### Rest use meta features

## Sentiment analysis
- Plane showing genres and sentiment scores

In [27]:
# Provides compound sentiment of a text based on VADER Sentiment analysis
# Input: string
# Output: compound sentiment between -1 and 1 (Negative/Positive)
def sentiment_scores(sentence):
    
    # Create a sentiment analysis object
    sent = SentimentIntensityAnalyzer()
    sent_dict = sent.polarity_scores(sentence)
    
#     print("Pos: {}, Neu: {}, Neg: {}, Score: {}".format(sent_dict['pos'],sent_dict['neu'],
#                                                        sent_dict['neg'],sent_dict['compound']))
    
    if sent_dict['compound']>=0.2:
        sent_dict['class'] = 'Positive'
    elif sent_dict['compound']<= -0.2:
        sent_dict['class'] = 'Negative'
    else:
        sent_dict['class'] = 'Neutral'
    
#     return [sentence, sent_dict['pos'],
#                 sent_dict['neu'], sent_dict['neg'],
#                 sent_dict['compound'],sent_dict['class']]
    return sent_dict['compound']


In [29]:
data['sentiment'] = data['lyrics_clean'].apply(lambda x: sentiment_scores(x))

In [30]:
data.head()

,track_name,track_artist,lyrics,track_popularity,track_album_name,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,tempo,duration_ms,language,song_year,trending_year,Weeks_On_Chart,Hit,clean,lyrics_clean,sentiment
1,i feel alive,steady rollin,"The trees, are singing in the wind The sky blu...",28,Love & Loss,Hard Rock Workout,rock,hard rock,0.303,0.880,...,135.225,373512,en,2017,NaN,0.0,0,"[tree, singing, wind, sky, blue, angel, smiled...",tree singing wind sky blue angel smiled saw lo...,0.9850
2,poison,bell biv devoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,Gold,"Back in the day - R&B, New Jack Swing, Swingbe...",r&b,new jack swing,0.845,0.652,...,111.904,262467,en,2005,1990.0,22.0,1,"[na, yeah, spyderman, freeze, full, effect, re...",na yeah spyderman freeze full effect ready ron...,-0.9968
3,baby it's cold outside (feat. christina aguilera),ceelo green,I really can't stay Baby it's cold outside I'v...,41,CeeLo's Magic Moment,Christmas Soul,r&b,neo soul,0.425,0.378,...,118.593,243067,en,2012,NaN,0.0,0,"[really, ca, stay, baby, cold, outside, got, g...",really ca stay baby cold outside got go away b...,0.9890
4,dumb litty,kard,Get up out of my business You don't keep me fr...,65,KARD 2nd Digital Single ‘Dumb Litty’,K-Party Dance Mix,pop,dance pop,0.760,0.887,...,130.018,193160,en,2019,NaN,0.0,0,"[get, business, keep, turning, 모두, 다, 여긴, witn...",get business keep turning 모두 다 여긴 witness 넌 바른...,-0.9993
5,soldier,james tw,"Hold your breath, don't look down, keep trying...",70,Chapters,urban contemporary,r&b,urban contemporary,0.496,0.639,...,147.764,224720,en,2019,NaN,0.0,0,"[hold, breath, look, keep, trying, darling, ok...",hold breath look keep trying darling okay scar...,0.9686


#### Helpful Links
- full end to end EDA and model no code: https://towardsdatascience.com/how-we-used-nltk-and-nlp-to-predict-a-songs-genre-from-its-lyrics-54e338ded537
- TF-IDF and Naive Bayes: https://iq.opengenus.org/naive-bayes-on-tf-idf-vectorized-matrix/
- Data cleaning- stemming, lemmtization: https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640 
- Feature Engineering: https://medium.com/@tanyufei0514/nlp-detecting-explicit-content-in-music-lyrics-7cd0450779e5
- Feature Engineering: https://towardsdatascience.com/sing-song-with-text-mining-cd8508ab51a5
- EDA, cosien similarity,speech tagging: https://towardsdatascience.com/visualizing-spotify-songs-with-python-an-exploratory-data-analysis-fc3fae3c2c09
- Embedding, Neural Nets, Julia project - https://spcman.github.io/getting-to-know-julia/deep-learning/nlp/song-lyrics-project/
- Topic modelling, EDA, feature engineering: https://www.datacamp.com/community/tutorials/R-nlp-machine-learning
- Keras/tensorflow: https://realpython.com/python-keras-text-classification/
- Smoothening: https://towardsdatascience.com/how-to-deal-with-imbalanced-data-in-python-f9b71aba53eb
